In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
trainImage_data_gen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.1  
)

In [ ]:

testImage_data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

In [ ]:
train_data_gen = trainImage_data_gen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='training',
    seed=0
)

In [ ]:
val_data_gen = testImage_data_gen.flow_from_directory(
    'train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='categorical',
    subset='validation',
    seed=0
)

# Retrieve class labels
labels = dict((v, k) for k, v in train_data_gen.class_indices.items())
print("Class labels:", labels)

In [ ]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"
feature_extractor = hub.KerasLayer(URL, input_shape=(224, 224, 3))
feature_extractor.trainable = False

# Build model
model = tf.keras.Sequential([
    feature_extractor,
    tf.keras.layers.Dense(len(labels), activation='softmax')  # number of classes
])

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

# Train model
history = model.fit(train_data_gen, epochs=20, validation_data=val_data_gen)

# Plotting training results
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(20)

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
test_x, test_y = val_data_gen.__getitem__(1)
prediction = model.predict(test_x)

# Display predictions vs. ground truth
plt.figure(figsize=(16, 16))
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.title(f'pred: {labels[np.argmax(prediction[i])]} / truth: {labels[np.argmax(test_y[i])]}')
    plt.imshow(test_x[i])

# Save the model
model.save('path/model.h5')

In [ ]:
loaded_model = tf.keras.models.load_model('path/model.h5')

In [ ]:
!tensorflowjs_converter --input_format=tf_saved_model --output_format=tfjs_graph_model --weight_shard_size_bytes=4194304 path/ tfjsmodel/